
# Image Display Widget using HTML5 Canvas Element

## Motivation

The HTML5 Canvas Element is a powerful tool for displaying resolution-dependent bitmap data.  It's
great for efficiently rendering 2D graphics, images, artwork, or game components.  My current
interest in the Canvas Element is to use it as the foundation for an interactive image viewer
within the IPython [Notebook](http://ipython.org/notebook.html) widget [framework](http://nbviewer.
ipython.org/github/ipython/ipython/blob/3.x/examples/Interactive%20Widgets/Index.ipynb).

I knew early on that I would need to learn a lot about JavaScript to make this project work. JavaScript
is a really curious programming language, where the basic language itself is a wilderness of
bizarre constructs and third-party modules.  Navigating through that mess is risky without a good reference: [book](https://play.google.com/store/books/details/Douglas_Crockford_JavaScript_The_Good_Parts?id=PXa2bby0oQ0C). I was really surprised at how much I had to learn about [RequireJS](http://requirejs.org/) and [BackboneJS](http://backbonejs.org/) before I could start making meaningful progress on the technical parts.  I found it particularly satisfying when the [Module Pattern](http://javascriptplayground.com/blog/2012/04/javascript-module-pattern/) finally started making sense to me.

## Design Thoughts

My plans for the Canvas Widget were to implement functionality required by my other projects.  These features includes:
  - Display an image via URL
  - Display an image from a Numpy array: gray scale, RGB, or RGBA
  - Accept user-defined callback functions to be called as response to mouse motion/click events
  - Support image zooming and panning via mouse

I wanted this widget to be as easy (or as hard??) to install and use as the built-in widgets.  Developing and installing widgets is made much easier now by using [jupyter-pip](https://github.com/jdfreder/jupyter-pip) extension to `setuptools`.  Also check out [ipython-widgetboilerplate](https://github.com/jdfreder/ipython-widgetboilerplate) for a comprehensive example on setting up a new widget project.  I found this [discussion](https://groups.google.com/d/msg/jupyter/A8dfJgRg4xM/wqNWAz1Lg00J) on the IPython email list very helpful when settting up my JavaScript module using RequireJS.
Motivation
Setting and manipulating the widget's image is done via numpy data arrays through a class property.  This part should not require additional tools.

Image data is synchronized between the Python back-end and JavaScript front-end as Base64-encoded compressed data.  Compression is performed using [imageio](https://github.com/imageio/imageio).  Options include lossly and lossless formats.


## Reference Information

Here is a list of helpful website I often browsed while learning new concepts and solving my implementation problems.

- Comprehensive Canvas reference at [whatwg.org](http://whatwg.org) specifically for developers: [The Canvas Element](http://developers.whatwg.org/the-canvas-element.html).
- JavaScript [Design Patterns](http://addyosmani.com/resources/essentialjsdesignpatterns/book/) book by Addy Osmani.
- Fundamental BackboneJS information:
  [View](http://backbonejs.org/#View), [Model](http://backbonejs.org/#Model), and
  [Events](http://backbonejs.org/#Events)

Here are some life-saving nuggets of BackboneJS information:

- StackOverflow discussion [rendering/appending views](http://stackoverflow.com/questions/9271507/how-to-render-and-append-sub-views-in-backbone-js)
- List of [built-in events](http://backbonejs.org/#Events-catalog)
- Nice example handling [mouse click events](http://danielarandaochoa.com/backboneexamples/blog/2012/02/28/handling-the-click-event-with-backbone/)
- Many more [BackboneJS examples](http://backbonejs.org/#examples)


## Lessons Learned

- In my application I don't have a clean correspondence between a Python back-end variable and a simple HTML5/JavaScript front-end widget, which is how the IPython built-in widgets are designed.  Instead I have an image and related cropping/scaling and offset parameters. I don't want to perform extra work and update all widget parameters when a single model parameter is changed. Some update are low overhead, e.g. redraw the image, while others require a lot of work, e.g. receive new PNG-compressed image data and copy it into the internal `Image` object.

- The best solution I've found to date is to forgo performing any work in my Backbone View's `this.update()` method, and instead use separate event handlers for each major part of my model.  These can be defined in `this.initialize()` and it might look like this:

    ```js
    this.model.on('change:_src', this.update_src, this);
    this.model.on('change:_width', this.update_width, this);
    this.model.on('change:_height', this.update_height, this);
    ```

- The method `this.update()` in my JavaScript derived class is called automatically by the `BackboneJS` framework for all `change` events triggered by IPython's Traitlet synchronization process.  This is implemented by the parent class `DOMWidgetView` via the method `initialize()` with a statement similar to the following:

    ```js
    this.model.on('change', this.update, this);
    ```

- In JavaScript I found I needed to call `this.update()` at the end of my `this.render()` method, even if my own `this.update()` doesn't do anything (or maybe doesn't even exist!).  If I left out this call odd things would happen, e.g. a second view of my model would not reflect any prior CSS style properties applied to earlier view instances.

- Here is how to check if a given variable exists inside the [Backbone Model](http://backbonejs.org/#Model-has):

    ```js
    if (this.model.has('_src')) {
      this.update_src();
    }
    ```

- The Canvas Element height and width properties were initially quite confusing for me.  That's because its a two-part problem! 1) the Canvas has an inherent width and height measured in data pixels; 2) the Canvas as displayed to the screen is controlled by the element's CSS width and height. The StackOverflow discussion [canvas-width-and-height-in-html5](http://stackoverflow.com/questions/4938346/canvas-width-and-height-in-html5) was extremely helpful.

- More on Canvas width and height: I found I could not rely on the style width and height to always default to the Canvas' inherent width and height properties after several changes. Maybe I did something wrong?  Maybe something to do with the IPython Notebook environment?  I don't know.  My solution was to explicitly set the Canvas style width and height whenever I needed to modify the Canvas' inherent width and height.

- Great description of the order of events when drawing an image to the canvas: http://www.w3.org/TR/2014/CR-2dcontext-20140821/#drawing-images-to-the-canvas.

- Understanding `Canvas Element` mouse [coordinates](http://www.html5canvastutorials.com/advanced/html5-canvas-mouse-coordinates/) and [events](http://stackoverflow.com/questions/10001283/html5-canvas-how-to-handle-mousedown-mouseup-mouseclick#) in the context of BackboneJS [event handling(http://danielarandaochoa.com/backboneexamples/blog/2012/02/28/handling-the-click-event-with-backbone/)

- The built-in IPython `DOMWidget` has methods for handling CSS styles.  Use those methods instead of building my own handlers for particular style properties, e.g. border thickness and color.

- The notebook about IPython's [Traitlet Events](http://nbviewer.ipython.org/github/ipython/ipython/blob/2.x/examples/Interactive%20Widgets/Widget%20Events.ipynb#Traitlet-Events) gives a lot of insight into the relationship between front-end widgets and back-end data structures.  IPython's built-in widgets generally connect a simple data type (an integer or float, a list of values, etc.) to a simple widget (slider, checkbox, dropdown list).  This explains why most built-in widgets focus on synchronizing a `value` property between Python and JavaScript.  The built-in button widget is slightly more complicated example. It doesn't manage any data, instead it generates `click` events, and it's framework is adapted to manage custom events.